In [1]:
import tkinter as tk
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load and configure the sentiment analysis model
max_words = 10000
max_sequence_length = 100
embedding_dim = 16

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_words, embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

tokenizer = Tokenizer(num_words=max_words)
tokenizer.word_index = {'<pad>': 0, 'the': 1, 'and': 2, 'a': 3, 'to': 4, 'of': 5}  # Replace with your word index if available

# Function to preprocess user reviews
def preprocess_reviews(reviews):
    sequences = tokenizer.texts_to_sequences(reviews)
    padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)
    return padded_sequences

# Function to predict sentiment using the AI model
def predict_sentiment(review):
    preprocessed_review = preprocess_reviews([review])
    sentiment_prediction = model.predict(preprocessed_review)[0]
    sentiment_label = "Positive" if sentiment_prediction >= 0.5 else "Negative"
    return sentiment_label, sentiment_prediction

def store_feedback(review):
    sentiment_label, sentiment_prediction = predict_sentiment(review)

    # Load existing feedback data from the Excel sheet
    feedback_df = pd.read_excel('feedback.xlsx')
    feedback_data = {'Review': [review], 'Sentiment': [sentiment_label], 'Prediction': [sentiment_prediction]}
    new_feedback_df = pd.DataFrame(feedback_data)

    # Append new feedback to the existing data
    feedback_df = pd.concat([feedback_df, new_feedback_df], ignore_index=True)

    # Save the updated feedback data to the Excel sheet
    feedback_df.to_excel('feedback.xlsx', index=False)

    print('Feedback received and stored successfully.')
    status_label.config(text='Thank you for your valuable feedback!')

    window.after(2000, window.destroy)  # Destroy the window after 2 seconds

def submit_review():
    review = review_entry.get()
    window.after(0, store_feedback, review)
    review_entry.delete(0, tk.END)
    submit_button.config(state=tk.DISABLED)  # Disable the submit button after submission


def show_thank_you_message():
    thank_you_label = tk.Label(window, text='Thank you for your valuable feedback!')
    thank_you_label.pack()


# Create the GUI window
window = tk.Tk()
window.title('Feedback Form')

# Create and position the widgets
review_label = tk.Label(window, text='Review:')
review_label.pack()

review_entry = tk.Entry(window, width=30)
review_entry.pack()

submit_button = tk.Button(window, text='Submit', command=submit_review)
submit_button.pack()

status_label = tk.Label(window, text='')
status_label.pack()

# Run the GUI event loop
window.mainloop()


1/1 [==============================] - 2s 2s/step
Feedback received and stored successfully.
